## Deliverable 2. Create a Customer Travel Destinations Map.

In [57]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [58]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("C:/Users\kkubi/Class Repo/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Maloy,NO,61.9353,5.1136,48.06,94,100,11.01,overcast clouds
1,1,Ulaanbaatar,MN,47.9077,106.8832,12.33,67,6,2.24,clear sky
2,2,Saint Anthony,US,45.0205,-93.2180,48.31,54,1,5.01,clear sky
3,3,Panaba,MX,21.2833,-88.2667,69.21,69,14,6.53,few clouds
4,4,Rikitea,PF,-23.1203,-134.9692,76.71,83,100,12.64,overcast clouds


In [59]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("On your skiing trip, what is the coldest temperature you want it to be? "))
max_temp = float(input("On your skiing trip, what is the hotest temperature you want it to be? " ))

On your skiing trip, what is the coldest temperature you want it to be? 15
On your skiing trip, what is the hotest temperature you want it to be? 32


In [60]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
#                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
16,16,Albany,US,42.6001,-73.9662,28.74,63,1,3.00,clear sky
22,22,Nikolskoye,RU,59.7035,30.7861,31.87,95,100,14.16,snow
23,23,Verkhniy Baskunchak,RU,48.2256,46.7217,30.15,81,100,12.19,overcast clouds
47,47,Narsaq,GL,60.9167,-46.0500,28.62,87,100,11.43,light snow
62,62,Qaqortoq,GL,60.7167,-46.0333,28.90,76,100,12.55,light snow
76,76,Pangnirtung,CA,66.1451,-65.7125,23.67,95,100,6.62,snow
92,92,Clyde River,CA,70.4692,-68.5914,17.83,86,100,26.46,snow
95,95,Broome,US,42.2506,-75.8330,27.66,81,1,1.01,clear sky
122,122,Jamestown,US,42.0970,-79.2353,31.59,80,1,3.00,clear sky
133,133,Gravdal,NO,68.1214,13.5031,31.80,66,94,24.02,overcast clouds


In [61]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                74
City                   74
Country                74
Lat                    74
Lng                    74
Max Temp               74
Humidity               74
Cloudiness             74
Wind Speed             74
Current Description    74
dtype: int64

In [62]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [63]:
preferred_cities_df.notnull().sum()

City_ID                74
City                   74
Country                74
Lat                    74
Lng                    74
Max Temp               74
Humidity               74
Cloudiness             74
Wind Speed             74
Current Description    74
dtype: int64

In [64]:
### 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
###       I did this step in STEP #3 by placing dropna() outside the loc method's square brackets.
### So, I didn't really have to do this here except to get a dataframe with the word "clean" in it :)
preferred_cities_clean_df = preferred_cities_df.dropna()
preferred_cities_clean_df.count()

City_ID                74
City                   74
Country                74
Lat                    74
Lng                    74
Max Temp               74
Humidity               74
Cloudiness             74
Wind Speed             74
Current Description    74
dtype: int64

In [65]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Albany,US,28.74,clear sky,42.6001,-73.9662,
22,Nikolskoye,RU,31.87,snow,59.7035,30.7861,
23,Verkhniy Baskunchak,RU,30.15,overcast clouds,48.2256,46.7217,
47,Narsaq,GL,28.62,light snow,60.9167,-46.0500,
62,Qaqortoq,GL,28.90,light snow,60.7167,-46.0333,
76,Pangnirtung,CA,23.67,snow,66.1451,-65.7125,
92,Clyde River,CA,17.83,snow,70.4692,-68.5914,
95,Broome,US,27.66,clear sky,42.2506,-75.8330,
122,Jamestown,US,31.59,clear sky,42.0970,-79.2353,
133,Gravdal,NO,31.80,overcast clouds,68.1214,13.5031,


In [66]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
         
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [68]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Albany,US,28.74,clear sky,42.6001,-73.9662,
22,Nikolskoye,RU,31.87,snow,59.7035,30.7861,Tourist House - Sablino
23,Verkhniy Baskunchak,RU,30.15,overcast clouds,48.2256,46.7217,Kombat
47,Narsaq,GL,28.62,light snow,60.9167,-46.0500,Sleep In Narsaq
62,Qaqortoq,GL,28.90,light snow,60.7167,-46.0333,Siniffik Inn
...,...,...,...,...,...,...,...
613,Baykit,RU,21.90,snow,61.6700,96.3700,
619,Tsagan Aman,RU,31.93,light snow,47.5639,46.7218,Tirupathi
624,Irbeyskoye,RU,31.51,overcast clouds,55.6428,95.4522,"Gostinitsa ""Agul"""
629,Kostomuksha,RU,25.41,light snow,64.5710,30.5767,U Mikhailicha


In [69]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.copy()
clean_hotel_df = clean_hotel_df.replace(r'^s*$', float('NaN'), regex = True) 
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
22,Nikolskoye,RU,31.87,snow,59.7035,30.7861,Tourist House - Sablino
23,Verkhniy Baskunchak,RU,30.15,overcast clouds,48.2256,46.7217,Kombat
47,Narsaq,GL,28.62,light snow,60.9167,-46.0500,Sleep In Narsaq
62,Qaqortoq,GL,28.90,light snow,60.7167,-46.0333,Siniffik Inn
76,Pangnirtung,CA,23.67,snow,66.1451,-65.7125,Kilabuk Lodge
92,Clyde River,CA,17.83,snow,70.4692,-68.5914,Qamaq Hotel
95,Broome,US,27.66,clear sky,42.2506,-75.8330,Chenango Valley State Park
122,Jamestown,US,31.59,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
133,Gravdal,NO,31.80,overcast clouds,68.1214,13.5031,Skreda Rorbusuiter
137,Zaysan,KZ,30.51,overcast clouds,47.4667,84.8667,Ерлан үй


In [70]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [71]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [72]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))